In [ ]:
#@title Import imports
import copy
import random
import time

import numpy

from functools import partial
from IPython.display import clear_output

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [ ]:
#@title Define the Ant Simulator (Environment)
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def if_then_else(condition, out1, out2):
    out1() if condition() else out2()

class AntSimulator(object):
    direction = ["north","east","south","west"]
    dir_row = [1, 0, -1, 0]
    dir_col = [0, 1, 0, -1]

    def __init__(self, max_moves):
        self.max_moves = max_moves
        self._moves = 0       
        self.eaten = 0
        self.routine = None   
        self.row_start = 0
        self.col_start = 0
        self.visualize=False  
        self.matrix = list()
        self.matrix_list = []

    def _reset(self):
        self.row = self.row_start 
        self.col = self.col_start 
        self.dir = 1
        #self.moves = 0 
        self._moves = 0        
        self.eaten = 0        
        self.matrix_exc = copy.deepcopy(random.choice(self.matrix_list))

    @property
    def position(self):
        return (self.row, self.col, self.direction[self.dir])

    @property
    def moves(self):
      return self._moves

    @moves.setter
    def moves(self, move):  
      self._moves = move
      if self.visualize:        
        self.render()
   
    def render(self):      
      print(f"Ant:{self.position}, moves:{self.moves}, eaten:{self.eaten} ")
      map = self.render_map(self.matrix_exc)
      clear_output()
      print(self.render_ant(map))
      time.sleep(1)
      pass

    def turn_left(self): 
        if self.moves < self.max_moves:
            self.moves += 1            
            self.dir = (self.dir - 1) % 4

    def turn_right(self):
        if self.moves < self.max_moves:
            self.moves += 1             
            self.dir = (self.dir + 1) % 4

    def move_forward(self):
        if self.moves < self.max_moves:
            self.moves += 1           
            self.row = (self.row + self.dir_row[self.dir]) % self.matrix_row
            self.col = (self.col + self.dir_col[self.dir]) % self.matrix_col
            if self.matrix_exc[self.row][self.col] == "food":
                self.eaten += 1               
                self.matrix_exc[self.row][self.col] = "empty"
            self.matrix_exc[self.row][self.col] = "passed"

    def sense_food(self):
        ahead_row = (self.row + self.dir_row[self.dir]) % self.matrix_row
        ahead_col = (self.col + self.dir_col[self.dir]) % self.matrix_col 
        return self.matrix_exc[ahead_row][ahead_col] == "food"

    def if_food_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_food, out1, out2)

    def run(self,routine):
        self._reset()
        while self.moves < self.max_moves:
            routine()

    def visualize_run(self, routine):
      self._reset()      
      while self.moves < self.max_moves:
        self.visualize=True
        routine()
      self.visualize = False

    def add_matrix(self, matrix):        
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == "#":
                    self.matrix[-1].append("food")
                elif col == ".":
                    self.matrix[-1].append("empty")
                elif col == "S":
                    self.matrix[-1].append("empty")
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.dir = 1
        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        #self.matrix_exc = copy.deepcopy(self.matrix) 
        self.matrix_list.append(self.matrix) 

    def clear_matrix(self):
      self.matrix_list.clear()      

    def render_map(self, matrix):
      map = ""
      for row in matrix:        
        map += self.render_row(row) + "\n"
      return map\

    def render_ant(self, map):
      idx = self.row * (self.matrix_col+1) + self.col 
      return map[:idx] + '☺' + map[idx+1:]
    
    def render_row(self, row):
      data = []
      for r in row:  
        if r == "food":
          data.append("#") 
        elif r == "passed":
          data.append("_") 
        elif r == "empty":
          data.append('.')
      return ''.join(data)

In [ ]:
#@title Defining the program function set
ant = AntSimulator(600)

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(ant.if_food_ahead, 2)
pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)
pset.addTerminal(ant.move_forward)
pset.addTerminal(ant.turn_left)
pset.addTerminal(ant.turn_right)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [ ]:
#@title Create the Toolbox and populate
toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalArtificialAnt(individual):
    # Transform the tree expression to functional Python code
    routine = gp.compile(individual, pset)
    # Run the generated routine
    ant.run(routine)
    return ant.eaten,

toolbox.register("evaluate", evalArtificialAnt)
toolbox.register("select", tools.selDoubleTournament, fitness_size=100, parsimony_size=2, fitness_first=True)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [ ]:
%%writefile santafe_trail1.txt
S###............................
...#............................
...#.....................###....
...#....................#....#..
...#....................#....#..
...####.#####........##.........
............#................#..
............#.......#...........
............#.......#........#..
............#.......#...........
....................#...........
............#................#..
............#...................
............#.......#.....###...
............#.......#..#........
.................#..............
................................
............#...........#.......
............#...#..........#....
............#...#...............
............#...#...............
............#...#.........#.....
............#..........#........
............#...................
...##. .#####....#...............
.#..............#...............
.#..............#...............
.#......#######.................
.#.....#........................
.......#........................
..####..........................
................................

Writing santafe_trail1.txt


In [ ]:
#@title Setup for Evolution
random.seed(222)

with  open("santafe_trail1.txt") as trail_file:
  ant.add_matrix(trail_file) 

pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

In [ ]:
#@title Run Evolution
GENERATIONS = 100 #@param {type:"slider", min:10, max:1000, step:5}
algorithms.eaSimple(pop, toolbox, 0.5, 0.2, GENERATIONS, stats, halloffame=hof) 

In [ ]:
#@title Show the Ant Running
# Run the generated routine  
best = hof[0]
routine = gp.compile(best, pset)
ant.visualize_run(routine)

In [ ]:
%%writefile santafe_trail2.txt
S###............................
...#............................
...#............................
...#............................
...#............................
...##########...................
............#...................
............#...................
............#...................
............#...................
............#...................
............#...................
............#...................
............#...................
............#########...........
............#########...........
............#########...........
............#########...........
............#...................
............#...................
............#...................
............#...................
............#...................
............#...................
.......######...................
.......#........................
.......#........................
.......########.................
................................
................................
................................
................................

Writing santafe_trail2.txt


In [ ]:
%%writefile santafe_trail3.txt
S...............................
................................
................................
................................
................................
................................
................................
............###############.....
............################....
............################....
............#...................
............########............
............#...................
............#...................
......###############...........
......###############...........
......#...........###...........
......###############...........
............#...................
............#...................
............#...................
............#...................
............#...................
............#...................
................................
................................
................................
................................
................................
................................
................................
................................

Writing santafe_trail3.txt


In [ ]:
#@title Load New Environments
ant.clear_matrix()  #clear out old environment
with  open("santafe_trail2.txt") as trail_file:
  ant.add_matrix(trail_file) 

with  open("santafe_trail3.txt") as trail_file:
  ant.add_matrix(trail_file) 

In [ ]:
#@title Test Best Ant on New Environments
ant.visualize_run(routine)

______._......_._______....._...
_......_......_._.._______.._...
_......_.______._☺___._.._.._...
_......_._......_..._._.._.._...
_.______._......_..._._______...
_._......_......________._......
_._......_......__.._.._._._____
_._......_________##_##_#_______
_._._______.#####_##________....
_._._....._.#####_#####_###_....
___._....._.#...._______..._.___
_..._....._._______#......._._..
_______..._._....._...______._..
__.._______._....._..._...._____
_______###______##_##._......_._
._...__###_#_##_##_##._......_._
._...__..._._.._.._##._.______._
._...______________##._._......_
.____________.._.______._......_
________.._.#.._._......_......_
_....___________._......_.______
_....___.._.#...._......_._.....
_....___.._.#...._.______._.....
_...._________..._._......_.....
_______..._.______._......_.....
__..._...._.__....._......______
___________.__....._._______....
__..._......__....._._....._....
__..._......________._....._....
__..._......_._...._._....._....
__..._.___

In [ ]:
#@title Train with All 3 Environments
ant.clear_matrix()  #clear out old environments

with  open("santafe_trail1.txt") as trail_file:
  ant.add_matrix(trail_file) 

with  open("santafe_trail2.txt") as trail_file:
  ant.add_matrix(trail_file) 

with  open("santafe_trail3.txt") as trail_file:
  ant.add_matrix(trail_file) 

In [ ]:
#@title Run Evolution - "Transfer Evolution"
GENERATIONS = 100 #@param {type:"slider", min:10, max:1000, step:5}
algorithms.eaSimple(pop, toolbox, 0.5, 0.2, GENERATIONS, stats, halloffame=hof)

#@title Show the Ant Running
# Run the generated routine  
best = hof[0]
routine = gp.compile(best, pset)
ant.visualize_run(routine)